In [127]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
import statsmodels.formula.api as smf

In [59]:
file_name = 'ConcessionSalesData_ForClass.xlsx'

df = pd.read_excel(file_name)
df.head()

,food_game,UserID,UseCount,revenue,game_week,special_discount,special_item,FAMILYGROUPNAME,Master_Item,MENUITEMNAME,...,first_week_discount,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Peanuts,Discount_Nachos,Discount_Pretzel,Discount_Popcorn,sth_rev_game,total_product_rev_nonSTH
0,BAG PEANUTS_Game 1,3304107,1,4.726207,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
1,BAG PEANUTS_Game 1,3405989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
2,BAG PEANUTS_Game 1,3302989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
3,BAG PEANUTS_Game 1,3253641,1,4.567500,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
4,BAG PEANUTS_Game 1,3315665,1,4.726615,Game 1,STH Discount Only,Yes,SNACKS,20500003,BAG PEANUTS,...,No Discount,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411


## Question 1

In [60]:
# Printing basic info 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19911 entries, 0 to 19910
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   food_game                 19911 non-null  object 
 1   UserID                    19911 non-null  int64  
 2   UseCount                  19911 non-null  int64  
 3   revenue                   19911 non-null  float64
 4   game_week                 19911 non-null  object 
 5   special_discount          19911 non-null  object 
 6   special_item              19911 non-null  object 
 7   FAMILYGROUPNAME           19911 non-null  object 
 8   Master_Item               19911 non-null  int64  
 9   MENUITEMNAME              19911 non-null  object 
 10  PRICES                    19911 non-null  float64
 11  actual_discount           19911 non-null  float64
 12  actual_price              19911 non-null  float64
 13  Discount Type             19911 non-null  object 
 14  Discou

## Promotional Items 
- Nachos- Souvenir popcorn (popcorn sold in a souvenir container)- Hog dog- Bag of Peanuts- Bavarian pretzel- Bottled water- Souvenir soda 32 oz (soda sold in a souvenir cup)

In [61]:
## Dropping Irrelevant Columns 
cols_to_drop = ['food_game', 'Master_Item', 'actual_price', 'actual_discount', 'first_week_discount']
df.drop(columns=cols_to_drop, axis=1, inplace=True)
df.head()

,UserID,UseCount,revenue,game_week,special_discount,special_item,FAMILYGROUPNAME,MENUITEMNAME,PRICES,Discount Type,Discount Percentage,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Peanuts,Discount_Nachos,Discount_Pretzel,Discount_Popcorn,sth_rev_game,total_product_rev_nonSTH
0,3304107,1,4.726207,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
1,3405989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
2,3302989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
3,3253641,1,4.567500,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
4,3315665,1,4.726615,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411


In [62]:
# Rename Some Columns for better code 
df.rename(columns={'MENUITEMNAME': 'item', 'total_product_rev_nonSTH': 'nonSTH_rev', 'PRICES': 'price'}, inplace=True)
df.head()

,UserID,UseCount,revenue,game_week,special_discount,special_item,FAMILYGROUPNAME,item,price,Discount Type,Discount Percentage,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Peanuts,Discount_Nachos,Discount_Pretzel,Discount_Popcorn,sth_rev_game,nonSTH_rev
0,3304107,1,4.726207,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
1,3405989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
2,3302989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
3,3253641,1,4.567500,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
4,3315665,1,4.726615,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411


In [63]:
# printing Special items and Items with price > 0
print(f"Orginal Data Size : {df.shape[0]}")
df_fil = df[(df.special_item == 'Yes') & (df.price > 0)]
print(f"Filtered Data Size : {df_fil.shape[0]}")

Orginal Data Size : 19911
Filtered Data Size : 12954


In [65]:
df_fil.head()

,UserID,UseCount,revenue,game_week,special_discount,special_item,FAMILYGROUPNAME,item,price,Discount Type,Discount Percentage,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Peanuts,Discount_Nachos,Discount_Pretzel,Discount_Popcorn,sth_rev_game,nonSTH_rev
0,3304107,1,4.726207,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
1,3405989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
2,3302989,1,4.730000,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
3,3253641,1,4.567500,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411
4,3315665,1,4.726615,Game 1,STH Discount Only,Yes,SNACKS,BAG PEANUTS,5.25,GA STM Discount,10,Yes,No,Yes,No,No,No,No,16441.58141,15296.65411


In [66]:
df_fil['Discount Percentage']

0        10
1        10
2        10
3        10
4        10
         ..
19906    50
19907    50
19908    50
19909    50
19910    50
Name: Discount Percentage, Length: 12954, dtype: int64

In [67]:
# Calculate actual_price based on the discount value 
df_fil['actual_price'] = df_fil['price'] * (1 - (df_fil['Discount Percentage'] /100))

In [69]:
# Remove the Free Items
df_fil = df_fil[df_fil.actual_price != 0]

In [70]:
# Printing unique promotional items 
df_fil.item.unique()

array(['BAG PEANUTS', 'BAVARIAN PRETZEL', 'BTL DEJA BLUE', 'HOT DOG',
       'NACHOS', 'SOUV CUP 32', 'SOUV POPCORN'], dtype=object)

In [71]:
item_map = {
    'BAG PEANUTS' : 'bag_of_peanuts',
    'BAVARIAN PRETZEL' : 'bavarian_pretzel',
    'BTL DEJA BLUE' : 'bottled_water',
    'HOT DOG' : 'hot_dog',
    'NACHOS' : 'nachos',
    'SOUV CUP 32' : 'souv_soda',
    'SOUV POPCORN' : 'souv_popcorn'
}

In [72]:
df_fil['item'] = df_fil['item'].apply(lambda x : item_map[x])

In [73]:
discount_cols = ['Discount_HotDog',
       'Discount_SouvCup', 'Discount_BtlWater', 'Discount_Peanuts',
       'Discount_Nachos', 'Discount_Pretzel', 'Discount_Popcorn']

In [74]:
# Visualising Discounted items for each week 
df_fil.groupby('game_week')[discount_cols].max()

,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Peanuts,Discount_Nachos,Discount_Pretzel,Discount_Popcorn
game_week,,,,,,,
Game 1,Yes,No,Yes,No,No,No,No
Game 2,No,No,No,Yes,Yes,No,No
Game 3,No,Yes,No,No,No,Yes,No
Game 4,Yes,No,Yes,No,No,No,No
Game 5,No,Yes,No,No,No,No,Yes
Game 6,No,No,Yes,No,No,Yes,No
Game 7,Yes,Yes,No,No,No,No,No
Game 8,Yes,Yes,Yes,No,No,No,Yes


In [75]:
## The userdemand depends on Factors like Discounted variables, nonStH_rev and Product
## Lets visualize in which scenarios there is maximum demand
agg_data = df_fil.groupby(['game_week', 'item', 'actual_price', 'nonSTH_rev',
                                        'Discount Percentage', 'Discount_HotDog',
                                        'Discount_SouvCup', 'Discount_BtlWater',
                                        'Discount_Peanuts', 'Discount_Nachos',
                                        'Discount_Pretzel', 'Discount_Popcorn'])['UseCount'].sum().reset_index()
#.sort_values(by='UseCount', ascending=False)
agg_data.iloc[:10]

,game_week,item,actual_price,nonSTH_rev,Discount Percentage,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Peanuts,Discount_Nachos,Discount_Pretzel,Discount_Popcorn,UseCount
0,Game 1,bag_of_peanuts,4.200,15296.65411,20,Yes,No,Yes,No,No,No,No,4
1,Game 1,bag_of_peanuts,4.725,15296.65411,10,Yes,No,Yes,No,No,No,No,101
2,Game 1,bavarian_pretzel,5.400,31998.73632,20,Yes,No,Yes,No,No,No,No,35
3,Game 1,bavarian_pretzel,6.075,31998.73632,10,Yes,No,Yes,No,No,No,No,188
4,Game 1,bottled_water,2.625,45272.22256,50,Yes,No,Yes,No,No,No,No,1017
5,Game 1,bottled_water,4.200,45272.22256,20,Yes,No,Yes,No,No,No,No,1
6,Game 1,bottled_water,4.725,45272.22256,10,Yes,No,Yes,No,No,No,No,1
7,Game 1,hot_dog,3.625,51002.55000,50,Yes,No,Yes,No,No,No,No,1064
8,Game 1,nachos,5.000,7989.81641,20,Yes,No,Yes,No,No,No,No,1
9,Game 1,nachos,5.625,7989.81641,10,Yes,No,Yes,No,No,No,No,70


In [17]:
# # Convert Categorical Variables in Discount as numerical
# tmp_data = agg_data.copy()

In [18]:
# discount_cols = [ col for col in agg.columns if 'Discount_' in col]
# discount_cols

In [19]:
# for col in discount_cols:
#     tmp_data[col] = tmp_data[col].map({'Yes':1, 'No':0})

In [76]:
## Calculating Price Elasticity of Different promotional Items based on aggregated data 
def fit_table(data, group_var='item', response_var='UseCount', predictor_vars=['actual_price', 'nonSTH_rev']):
    results = {}
    for item, group_data in data.groupby(group_var):
        # Log-transform the specified variables
        print(f"Item:{item}")
        
        X = np.log(group_data[predictor_vars])
        y = np.log(group_data[response_var])
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()
        results[item] = model
    
    return results

In [77]:
results = fit_table(data = agg_data)

Item:bag_of_peanuts
Item:bavarian_pretzel
Item:bottled_water
Item:hot_dog
Item:nachos
Item:souv_popcorn
Item:souv_soda


In [78]:
for item, res in results.items():
    print("########################################")
    print(f"Item:{item}")

    print(res.summary())
    print()

########################################
Item:bag_of_peanuts
                            OLS Regression Results                            
Dep. Variable:               UseCount   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.226
Method:                 Least Squares   F-statistic:                   0.07727
Date:                Sat, 24 Feb 2024   Prob (F-statistic):              0.926
Time:                        00:11:46   Log-Likelihood:                -21.429
No. Observations:                  11   AIC:                             48.86
Df Residuals:                       8   BIC:                             50.05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [79]:
summary = summary_col(list(results.values()),  model_names=list(results.keys()), info_dict={'R2': lambda x: f"{x.rsquared:.2f}"})
summary

,bag_of_peanuts,bavarian_pretzel,bottled_water,hot_dog,nachos,souv_popcorn,souv_soda
const,-9.5916,-54.2666,38.1732,-37.8583,-31.4986,9.2933,10.0793
,(35.4148),(38.4236),(17.0028),(69.6815),(50.3625),(8.9698),(31.0069)
actual_price,0.1373,-0.7752,-5.2380,-3.6062,-0.1560,-4.1485,-2.5419
,(3.7779),(2.0753),(1.8766),(1.6630),(4.0851),(1.0712),(1.1693)
nonSTH_rev,1.3631,5.7844,-2.5859,4.5500,3.8443,0.2819,-0.0349
,(3.5531),(3.4510),(1.6127),(6.3661),(5.3460),(0.9250),(3.0101)
R-squared,0.0190,0.4285,0.4357,0.5419,0.0586,0.6620,0.4001
R-squared Adj.,-0.2263,0.3246,0.3489,0.4401,-0.1297,0.6005,0.2668
R2,0.02,0.43,0.44,0.54,0.06,0.66,0.40


In [24]:
# Creating Results DataFrame 
item_list = ['bag_of_peanuts', 'bavarian_pretzel', 'bottled_water', 'hot_dog', 'nachos', 'souv_popcorn', 'souv_soda']
const_list = [-9.5916,-54.2666,38.1732,-37.8583,-31.4986,9.2933,10.0793]
actual_price_list = [0.1373,-0.7752,-5.2380,-3.6062,-0.1560,-4.1485,-2.5419]
nonSTH_rev_list = [1.3631,5.7844,-2.5859,4.5500,3.8443,0.2819,-0.0349]

result_df = pd.DataFrame(
    {
        "item" : item_list,
        "Const" : const_list,
        "ActualPrice" : actual_price_list,
        'nonSTH_rev' : nonSTH_rev_list
    }
)

result_df

,item,Const,ActualPrice,nonSTH_rev
0,bag_of_peanuts,-9.5916,0.1373,1.3631
1,bavarian_pretzel,-54.2666,-0.7752,5.7844
2,bottled_water,38.1732,-5.2380,-2.5859
3,hot_dog,-37.8583,-3.6062,4.5500
4,nachos,-31.4986,-0.1560,3.8443
5,souv_popcorn,9.2933,-4.1485,0.2819
6,souv_soda,10.0793,-2.5419,-0.0349


In [83]:

min_discount_per_user = df.groupby('UserID')['Discount Percentage'].min().reset_index()

user_counts = min_discount_per_user['Discount Percentage'].value_counts().reset_index()
user_counts.rename(columns={'count' : "STH_USERS"}, inplace=True)
user_counts

,Discount Percentage,STH_USERS
0,10,2684
1,50,838
2,20,672
3,100,1


In [85]:
ten_perc_dis_users = 2684 / (2684 + 672)
t20_perc_dis_users = 672 / (2684 + 672)

ten_perc_dis_users, t20_perc_dis_users


(0.799761620977354, 0.20023837902264602)

In [92]:
## 50 % discount are available to all the STH users
## 10 and 20 % discount depends on the club level. So to analyse the affect of STH levels, irrespective of clubs
## Lets adjust the sales to consider 100% STH available for each category 

## 10% discount consists of 80% STH -> To Upsample it to 100 , increase it by 25 % 
## 20% discounts consists of 20% STH -> to Upsample it to 100, increase it 400%

agg_data2 = agg_data.copy()
updated_use_count = []
for idx,row in agg_data2.iterrows():
    if row['Discount Percentage'] == 10:
        updated_use_count.append(1.25 * row['UseCount'])
    elif row['Discount Percentage'] == 20:
        updated_use_count.append(5 * row['UseCount'])
    else:
        updated_use_count.append(row['UseCount'])

agg_data2['UseCount'] = updated_use_count

In [93]:
agg_data2

,game_week,item,actual_price,nonSTH_rev,Discount Percentage,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Peanuts,Discount_Nachos,Discount_Pretzel,Discount_Popcorn,UseCount
0,Game 1,bag_of_peanuts,4.200,15296.654110,20,Yes,No,Yes,No,No,No,No,20.00
1,Game 1,bag_of_peanuts,4.725,15296.654110,10,Yes,No,Yes,No,No,No,No,126.25
2,Game 1,bavarian_pretzel,5.400,31998.736320,20,Yes,No,Yes,No,No,No,No,175.00
3,Game 1,bavarian_pretzel,6.075,31998.736320,10,Yes,No,Yes,No,No,No,No,235.00
4,Game 1,bottled_water,2.625,45272.222560,50,Yes,No,Yes,No,No,No,No,1017.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Game 8,bottled_water,2.625,23320.500000,50,Yes,Yes,Yes,No,No,No,Yes,470.00
88,Game 8,hot_dog,3.625,44715.750000,50,Yes,Yes,Yes,No,No,No,Yes,806.00
89,Game 8,nachos,5.625,7578.905183,10,Yes,Yes,Yes,No,No,No,Yes,57.50
90,Game 8,souv_popcorn,5.625,4859.000000,50,Yes,Yes,Yes,No,No,No,Yes,86.00


In [94]:
# Now Calculate Adjusted Price Elasticity
results_adj = fit_table(data = agg_data2)
summary_adj = summary_col(list(results_adj.values()),  model_names=list(results_adj.keys()), info_dict={'R2': lambda x: f"{x.rsquared:.2f}"})
summary_adj

Item:bag_of_peanuts
Item:bavarian_pretzel
Item:bottled_water
Item:hot_dog
Item:nachos
Item:souv_popcorn
Item:souv_soda


,bag_of_peanuts,bavarian_pretzel,bottled_water,hot_dog,nachos,souv_popcorn,souv_soda
const,-9.3066,-23.5499,37.5866,-21.3572,-19.8468,14.4199,1.1133
,(22.5361),(13.3488),(13.9801),(28.9000),(32.6726),(7.5002),(9.4883)
actual_price,0.0790,-1.3452,-4.1444,-2.6079,-0.7236,-3.8445,-1.4216
,(2.4041),(0.7210),(1.5430),(0.6897),(2.6502),(0.8957),(0.3578)
nonSTH_rev,1.4038,2.9927,-2.6067,2.9104,2.7212,-0.3289,0.6853
,(2.2610),(1.1989),(1.3260),(2.6403),(3.4682),(0.7735),(0.9211)
R-squared,0.0491,0.7621,0.4513,0.7744,0.0891,0.6692,0.6570
R-squared Adj.,-0.1886,0.7189,0.3669,0.7242,-0.0930,0.6090,0.5807
R2,0.05,0.76,0.45,0.77,0.09,0.67,0.66


## Question 2

#### Calculating Casual Effect of each Promotional Item on others

In [ ]:
# Price Elasticity 

In [142]:
def calculate_casual_effect(data, target_item, discount_col):
    data2 = data[data['item'] != target_item]
    summary_df = pd.DataFrame()
    # Iterate over each unique item in the 'item' column
    for item in data2['item'].unique():
        # Subset the data for the current item
        item_data = data2[data2['item'] == item]
        
        # Fit the linear regression model
        model = smf.ols(f'np.log(UseCount) ~ np.log(actual_price) + np.log(nonSTH_rev) + {discount_col}', data=item_data).fit()
        
        # Extract the summary statistics for the model
        summary = model.summary2().tables[1]
        
        # Add a column with the item name
        summary['item'] = item
    
        tmp = pd.DataFrame(summary.T.reset_index().iloc[0][1:]).T
        tmp['item'] = item
        #tmp.drop('index', inplace=True, axis=0)
        # Append the summary statistics to the summary_df DataFrame
        summary_df = pd.concat([summary_df,tmp])
    
    # Use the tabulate library to create a markdown table
    markdown_table = tabulate(summary_df, headers='keys', tablefmt='pipe', showindex=False)
    return markdown_table

In [143]:
# Casual Effect of HotDog
print(calculate_casual_effect(data=agg_data2, target_item='hot_dog',discount_col='Discount_HotDog'))

|   Intercept |   Discount_HotDog[T.Yes] |   np.log(actual_price) |   np.log(nonSTH_rev) | item             |
|------------:|-------------------------:|-----------------------:|---------------------:|:-----------------|
|  -14.0272   |               -0.634657  |               0.794083 |             1.83233  | bag_of_peanuts   |
|  -31.0695   |                0.354961  |              -1.44105  |             3.71297  | bavarian_pretzel |
|   33.6592   |               -1.48808   |              -4.79059  |            -2.06183  | bottled_water    |
|  -19.2033   |                0.054161  |              -0.79503  |             2.65916  | nachos           |
|   19.6617   |               -0.308547  |              -4.15333  |            -0.867646 | souv_popcorn     |
|    0.840362 |               -0.0214514 |              -1.42646  |             0.712895 | souv_soda        |


In [144]:
# Casual Effect of bag_of_peanuts
print(calculate_casual_effect(data=agg_data2, target_item='bag_of_peanuts', discount_col='Discount_Peanuts'))

|   Intercept |   Discount_Peanuts[T.Yes] |   np.log(actual_price) |   np.log(nonSTH_rev) | item             |
|------------:|--------------------------:|-----------------------:|---------------------:|:-----------------|
|   -22.4146  |                -0.269597  |               -1.31546 |             2.88224  | bavarian_pretzel |
|    45.0247  |                 1.93653   |               -4.75496 |            -3.26777  | bottled_water    |
|   -22.5065  |                 0.0209145 |               -2.60126 |             3.01655  | hot_dog          |
|   -34.5919  |                11.8307    |               17.8346  |             0.860657 | nachos           |
|    12.6093  |                -0.425092  |               -3.62389 |            -0.159206 | souv_popcorn     |
|    -6.20674 |                -0.436772  |               -1.3442  |             1.37058  | souv_soda        |


In [145]:
# Casual Effect of bavarian_pretzel
print(calculate_casual_effect(data=agg_data2, target_item='bavarian_pretzel', discount_col='Discount_Pretzel'))

|   Intercept |   Discount_Pretzel[T.Yes] |   np.log(actual_price) |   np.log(nonSTH_rev) | item           |
|------------:|--------------------------:|-----------------------:|---------------------:|:---------------|
|    -9.16218 |                 -0.106148 |               0.120878 |              1.38492 | bag_of_peanuts |
|    36.1749  |                  1.0012   |              -3.98983  |             -2.50884 | bottled_water  |
|   -27.6593  |                 -0.199088 |              -2.34187  |              3.46529 | hot_dog        |
|   -26.7139  |                 -0.660907 |              -0.253973 |              3.43011 | nachos         |
|    22.8789  |                  0.625487 |              -4.68358  |             -1.15984 | souv_popcorn   |
|    -2.49593 |                  0.292123 |              -1.48263  |              1.02919 | souv_soda      |


In [146]:
# Casual Effect of bottled_water
print(calculate_casual_effect(data=agg_data2, target_item='bottled_water', discount_col='Discount_BtlWater'))

|   Intercept |   Discount_BtlWater[T.Yes] |   np.log(actual_price) |   np.log(nonSTH_rev) | item             |
|------------:|---------------------------:|-----------------------:|---------------------:|:-----------------|
|   -9.06243  |                 -0.970703  |               0.830969 |             1.32594  | bag_of_peanuts   |
|  -21.3805   |                  0.197502  |              -1.4347   |             2.78859  | bavarian_pretzel |
|  -19.6004   |                  0.0983871 |              -2.54533  |             2.7326   | hot_dog          |
|  -19.5845   |                  0.0566604 |              -0.781381 |             2.69891  | nachos           |
|   15.0163   |                 -0.0348451 |              -3.8796   |            -0.390222 | souv_popcorn     |
|   -0.977328 |                  0.218236  |              -1.66035  |             0.909872 | souv_soda        |


In [148]:
# Casual Effect of nachos
print(calculate_casual_effect(data=agg_data2, target_item='nachos', discount_col='Discount_Nachos'))

|   Intercept |   Discount_Nachos[T.Yes] |   np.log(actual_price) |   np.log(nonSTH_rev) | item             |
|------------:|-------------------------:|-----------------------:|---------------------:|:-----------------|
|   -39.1977  |               11.7189    |               19.2977  |             1.47056  | bag_of_peanuts   |
|   -22.4146  |               -0.269597  |               -1.31546 |             2.88224  | bavarian_pretzel |
|    45.0247  |                1.93653   |               -4.75496 |            -3.26777  | bottled_water    |
|   -22.5065  |                0.0209145 |               -2.60126 |             3.01655  | hot_dog          |
|    12.6093  |               -0.425092  |               -3.62389 |            -0.159206 | souv_popcorn     |
|    -6.20674 |               -0.436772  |               -1.3442  |             1.37058  | souv_soda        |


In [149]:
# Casual Effect of souv_popcorn
print(calculate_casual_effect(data=agg_data2, target_item='souv_popcorn', discount_col='Discount_Popcorn'))

|   Intercept |   Discount_Popcorn[T.Yes] |   np.log(actual_price) |   np.log(nonSTH_rev) | item             |
|------------:|--------------------------:|-----------------------:|---------------------:|:-----------------|
|   -18.73    |                 1.0258    |              -0.159259 |             2.42634  | bag_of_peanuts   |
|   -37.1519  |                -0.466081  |              -0.550729 |             4.18807  | bavarian_pretzel |
|    45.4748  |                -0.826349  |              -4.30046  |            -3.33264  | bottled_water    |
|   -20.4667  |                 0.0805323 |              -2.6206   |             2.82714  | hot_dog          |
|   -30.2614  |                 0.595731  |              -0.698245 |             3.87264  | nachos           |
|     0.19441 |                 0.0634243 |              -1.3777   |             0.763865 | souv_soda        |


In [150]:
# Casual Effect of souv_soda
print(calculate_casual_effect(data=agg_data2, target_item='souv_soda', discount_col='Discount_SouvCup'))

|   Intercept |   Discount_SouvCup[T.Yes] |   np.log(actual_price) |   np.log(nonSTH_rev) | item             |
|------------:|--------------------------:|-----------------------:|---------------------:|:-----------------|
|    -11.7928 |                0.870149   |              -0.773075 |             1.7686   | bag_of_peanuts   |
|    -23.2886 |               -0.154797   |              -1.35245  |             2.97614  | bavarian_pretzel |
|     29.0217 |                0.562857   |              -4.36129  |            -1.77518  | bottled_water    |
|    -21.2035 |                0.00639095 |              -2.6101   |             2.89604  | hot_dog          |
|    -19.886  |               -0.244724   |              -0.486974 |             2.69475  | nachos           |
|     15.7318 |                0.124681   |              -3.80188  |            -0.506879 | souv_popcorn     |


In [151]:
agg_data2.item.unique()

array(['bag_of_peanuts', 'bavarian_pretzel', 'bottled_water', 'hot_dog',
       'nachos', 'souv_popcorn', 'souv_soda'], dtype=object)

In [152]:
agg_data2.head()

,game_week,item,actual_price,nonSTH_rev,Discount Percentage,Discount_HotDog,Discount_SouvCup,Discount_BtlWater,Discount_Peanuts,Discount_Nachos,Discount_Pretzel,Discount_Popcorn,UseCount
0,Game 1,bag_of_peanuts,4.200,15296.65411,20,Yes,No,Yes,No,No,No,No,20.00
1,Game 1,bag_of_peanuts,4.725,15296.65411,10,Yes,No,Yes,No,No,No,No,126.25
2,Game 1,bavarian_pretzel,5.400,31998.73632,20,Yes,No,Yes,No,No,No,No,175.00
3,Game 1,bavarian_pretzel,6.075,31998.73632,10,Yes,No,Yes,No,No,No,No,235.00
4,Game 1,bottled_water,2.625,45272.22256,50,Yes,No,Yes,No,No,No,No,1017.00


In [153]:
item_tups = [('bag_of_peanuts','Discount_Peanuts'), 
             ('bavarian_pretzel', 'Discount_Pretzel'),
             ('bottled_water','Discount_BtlWater'),
             ('hot_dog','Discount_HotDog'),
             ('nachos','Discount_Nachos'),
             ('souv_popcorn','Discount_Popcorn'),
             ('souv_soda','Discount_SouvCup')
            ]

In [162]:
names =['bag_of_peanuts', 'bavarian_pretzel', 'bottled_water', 'hot_dog',
       'nachos', 'souv_popcorn', 'souv_soda']
cf = pd.DataFrame(index=names, columns=names)
pv = pd.DataFrame(index=names, columns=names)

# Iterate over each item name
for n , dis in item_tups:
    # Filter out the current item from the data
    print(n,dis)
    ac3 = agg_data2[agg_data2['item'] != n]

    # Define the formula for the regression model
    formula = f'np.log(UseCount) ~ np.log(actual_price) + np.log(nonSTH_rev) + {dis}'

    # Fit the linear regression model for each item
    fits = {item: smf.ols(formula, data=ac3[ac3['item'] == item]).fit() for item in ac3['item'].unique()}

    # Extract coefficients and p-values for the Discount term
    for item, fit in fits.items():
        if item != n:
            cf.loc[item, n] = fit.params[f'{dis}[T.Yes]']
            pv.loc[item, n] = fit.pvalues[f'{dis}[T.Yes]']

# Replace NaN with appropriate values if necessary
cf.fillna(value=np.nan, inplace=True)
pv.fillna(value=np.nan, inplace=True)

# Print the coefficients and p-values DataFrames
print(cf)
#print(pv)

bag_of_peanuts Discount_Peanuts
bavarian_pretzel Discount_Pretzel
bottled_water Discount_BtlWater
hot_dog Discount_HotDog
nachos Discount_Nachos
souv_popcorn Discount_Popcorn
souv_soda Discount_SouvCup
                  bag_of_peanuts  bavarian_pretzel  bottled_water   hot_dog  \
bag_of_peanuts               NaN         -0.106148      -0.970703 -0.634657   
bavarian_pretzel       -0.269597               NaN       0.197502  0.354961   
bottled_water           1.936534          1.001202            NaN -1.488080   
hot_dog                 0.020914         -0.199088       0.098387       NaN   
nachos                 11.830687         -0.660907       0.056660  0.054161   
souv_popcorn           -0.425092          0.625487      -0.034845 -0.308547   
souv_soda              -0.436772          0.292123       0.218236 -0.021451   

                     nachos  souv_popcorn  souv_soda  
bag_of_peanuts    11.718912      1.025803   0.870149  
bavarian_pretzel  -0.269597     -0.466081  -0.154797  


In [166]:
## Row Represent of Items
## Cols represent how diff items causes discounts on row items
cf

,bag_of_peanuts,bavarian_pretzel,bottled_water,hot_dog,nachos,souv_popcorn,souv_soda
bag_of_peanuts,NaN,-0.106148,-0.970703,-0.634657,11.718912,1.025803,0.870149
bavarian_pretzel,-0.269597,NaN,0.197502,0.354961,-0.269597,-0.466081,-0.154797
bottled_water,1.936534,1.001202,NaN,-1.488080,1.936534,-0.826349,0.562857
hot_dog,0.020914,-0.199088,0.098387,NaN,0.020914,0.080532,0.006391
nachos,11.830687,-0.660907,0.056660,0.054161,NaN,0.595731,-0.244724
souv_popcorn,-0.425092,0.625487,-0.034845,-0.308547,-0.425092,NaN,0.124681
souv_soda,-0.436772,0.292123,0.218236,-0.021451,-0.436772,0.063424,NaN
